In [1]:
import os, subprocess, requests, json
import pandas as pd, numpy as np
from datetime import datetime

In [2]:
year = datetime.now().year
dates, weekdays, weeks = [], [], []
relevant_weekdays = [5,6]
saison_start = datetime(year,4,1)
saison_ende = datetime(year,10,31)
weekday_dict = {5:'Samstag',6:'Sonntag'}
file_name = f'NBDF_schleppwochenenden_{year}.xlsx'
sub_dir = 'schleppkalender'
feiertag_api = f'https://feiertage-api.de/api/?jahr={year}'
county = 'BY'

In [3]:
feiertage_obj = json.loads(requests.get(feiertag_api).content)[county]

In [4]:
feiertage_dict = dict()
for key, val in feiertage_obj.items():
    date = datetime.strptime(val['datum'], '%Y-%m-%d')
    skip = len(val['hinweis'])>0
    name = key
    if not skip:
        feiertage_dict[date] = name
feiertage = list(feiertage_dict.keys())

In [5]:
for month in range(1,13,1):
    for day in range(1,32,1):
        try:
            date = datetime(year=year,month=month,day=day)
            weekday = datetime.weekday(date)
            week = date.strftime("%V")
            if date > saison_start and date < saison_ende:
                if date in feiertage:
                    weekdays.append(feiertage_dict[date])
                else:
                    weekdays.append(weekday_dict[weekday])
                weeks.append(week)
                dates.append(date.strftime('%d.%m.%Y'))
        except:
            continue

In [6]:
abs_path = os.path.join(os.getcwd(),sub_dir)
if not os.path.exists(abs_path):
    os.makedirs(abs_path)

In [7]:
weekend_dict = {
    'KW':weeks,
    'Datum':dates,
    'Wochentag':weekdays,
    'Verfügbar als WF':[' ' for _ in range(len(dates))],
    'Verfügbar als Helfer':[' ' for _ in range(len(dates))],
    'Kann ich auf keinen Fall':[' ' for _ in range(len(dates))]
}

In [8]:
df_blank = pd.DataFrame(weekend_dict)

In [9]:
avaliable_days = 12
avaliable_helper = 4
non_avaliable_days = 6
indexes = [idx for idx in range(len(dates))]
selection = list(np.random.choice(indexes,size=avaliable_days+avaliable_helper+non_avaliable_days,replace=False))
for idx in selection[:avaliable_days]:
    weekend_dict['Verfügbar als WF'][idx] = 'X'
for idx in selection[avaliable_days:avaliable_days+avaliable_helper]:
    weekend_dict['Verfügbar als Helfer'][idx] = 'X'
for idx in selection[avaliable_days+avaliable_helper:]:
    weekend_dict['Kann ich auf keinen Fall'][idx] = 'X'

In [10]:
df_example = pd.DataFrame(weekend_dict)

In [11]:
with pd.ExcelWriter(os.path.join(abs_path,file_name)) as writer:  # doctest: +SKIP
    df_blank.to_excel(writer, sheet_name=str(year),index=False)
    df_example.to_excel(writer, sheet_name='Beispiel',index=False)

In [12]:
subprocess.Popen(['xdg-open', abs_path]);

In [13]:
print('All Done.')

All Done.
